In [1]:
########
# LC generator for T2Ceps
########

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
bg_color = "#000000"
use_period_color = True
R_color = 1
G_color = 1
B_color = 1
var_Type = "t2cep"
pltSIZE = 128
pltDPI = 72
stackSize = 2500
generateClones = True

webaddress_base = "http://www.astrouw.edu.pl/ogle/ogle4/OCVS/lmc/" + var_Type + "/"
web_data_folder = "phot/I/OGLE-LMC-" + var_Type.upper() + "-"

ident = pd.read_csv(webaddress_base + "ident.dat", header=None, delim_whitespace=True, usecols=[0,3])
ident.columns = ['id','type']

datacount = ident.shape[0]
cloneNumber = stackSize-datacount

t2cep_data = pd.read_csv(webaddress_base + "t2cep.dat", header=None, delim_whitespace=True, usecols=[0,3,5])

t2cep_data.columns = ['id','period','T_max']
t2cep_data['id'] = pd.Series(t2cep_data['id']).str[15:20]
t2cep_data.set_index('id', inplace=True)

#rrlyr_data['id'] = pd.to_numeric(rrlyr_data['id'])
#rrlyr_data = rrlyr_data.to_numpy(copy=True)

In [3]:
print("Available data:", datacount)
print("Desired number of data:", stackSize)
print("Will generate", cloneNumber , "clones...")

Available data: 290
Desired number of data: 2500
Will generate 2210 clones...


In [ ]:
webaddress = webaddress_base + web_data_folder

if generateClones:
    cloneCounter = 0
    parentCounter = 0

for row in range(datacount):
#    if ident['type'][rrlrow] != "RRd":
#        continue
#    else:
    star_identifier = str(f'{row+1:03d}')
    data_filename = webaddress + star_identifier + ".dat"

    try:
        data = pd.read_csv(data_filename, delim_whitespace=True, header=None, usecols=[0,1,2])
    except:
        print("No variable found at " + data_filename)
        continue
    data.columns = ["date", "mag", "error"]
                
    T_max = np.float(t2cep_data.loc[star_identifier].T_max)
    period = np.float(t2cep_data.loc[star_identifier].period)
    print(T_max, "--", period)

    if T_max > data['date'][0]:
        t_null = data['date'][0]
        pr = np.ceil((T_max - t_null)/period)
#         print(pr)
        T_max = T_max - pr*period
#         print(T_max)
              
    if use_period_color:
#         print("Color coded period")
        if 0 < period and period <=1:
            R_color = period
            G_color = 0
            B_color = 0
        if 1 < period and period <=10:
            R_color = 1
            G_color = period/10
            B_color = 0
        if 10 < period and period <=100:
            R_color = 1
            G_color = 1
            B_color = period/100
        if period > 100:
            R_color = 1
            G_color = 1
            B_color = 1
    else:
        print("Not using color coded period")
    
#     print("Color: R:", R_color, "G:", G_color, "B:", B_color)
#     print(T_max, "--", period)
    
    flux_cont = []
    flux_norm_cont = []
    phase_cont = []
    
    mag_cont = []
    t_cont = []
    error_cont = []

    for i in data.index:
        mag = data['mag'][i]
        t = data['date'][i]
        error = data['error'][i]
        mag_cont.append(mag)
        t_cont.append(t)
        error_cont.append(error)
        
        phase = (t-T_max)/period - int((t-T_max)/period)
        flux = 10**(-0.4*(mag-25))
        flux_cont.append(flux)
        phase_cont.append(phase)
        #print(mag , "--",  flux)
   
    if generateClones:
        if row == 0:
            cloneGenRatio = round((stackSize-datacount)/datacount)
            floatRatio = (stackSize-datacount)/datacount
        else:
            # Checking and adjusting clone number
            remParent = datacount-parentCounter
            remAll = (stackSize-cloneCounter-parentCounter)-(datacount-parentCounter)
            cloneGenRatio = round(remAll/remParent)
            floatRatio = remAll/remParent
        
        print("===================")
        max_error = max(error_cont)
        print("Max error:", max_error)
        error_cont = np.random.normal(0, max_error/3, len(error_cont))
        print("Error array generated for " + var_Type + "_" + star_identifier)
         
        for cl in range(cloneGenRatio):
            flux_clone_cont = []
            flux_clone_norm_cont = []
            rand_error_array = np.random.choice(error_cont,len(error_cont))
            mag_clone_cont = mag_cont + rand_error_array
            
            print("Clone generation", cl)
            for c in range(len(mag_clone_cont)):
                flux_clone = 10**(-0.4*(mag_clone_cont[c]-25))
                flux_clone_cont.append(flux_clone)
            
            max_clone_flux = max(flux_clone_cont)
            for i in flux_clone_cont:
                flux_clone_norm = (i/max_clone_flux)
                flux_clone_norm_cont.append(flux_clone_norm)

            phase_cont = np.array(phase_cont)
            flux_clone_norm_cont = np.array(flux_clone_norm_cont)
            
            #     
            # Plotting the folded lightcurve for cloned variable
            # 
            plt.figure(num=None, figsize=(pltSIZE/pltDPI, pltSIZE/pltDPI), dpi=pltDPI, facecolor=bg_color)

            plt.plot(phase_cont, flux_clone_norm_cont, color=(R_color,G_color,B_color), marker='.', linestyle='none', markersize=0.7)
            plt.plot(phase_cont+1, flux_clone_norm_cont, color=(R_color,G_color,B_color), marker='.', linestyle='none', markersize=0.7)

            plt.axis('off')

        #     period_str = str(period).replace(".","_")
        #     period_str = period_str[0:6]
        #     filename = "T2CEP_LCs/" + var_Type + "_" + star_identifier + ".png"
            filename = "../testdata_color/" + var_Type + "_" + star_identifier + "_clone_" + str(cl) +".png"

        #     filename = "T2CEP_LCs/" + period_str + "_" + var_Type + "_" + star_identifier + ".png"
        #     filename = "T2CEP_LCs/" + period_str + ".png"
            print("Saving " + filename)
            plt.tight_layout(0)
            plt.gcf().savefig(filename, bbox_inches=None, pad_inches=0, facecolor=bg_color, edgecolor='none', dpi='figure')
#             plt.show()
            plt.close()
            
            cloneCounter += 1
        
            ###     
            # End of plotting for the clone(s)
            ### 

# 
# Creating data for the original variable star
# 
    print(len(flux_cont))
    max_flux = max(flux_cont)
        
    for i in flux_cont:
        flux_norm = (i/max_flux)
        flux_norm_cont.append(flux_norm)

    #if len(phase_cont) == len(flux_norm_cont):
    phase_cont = np.array(phase_cont)
    flux_norm_cont = np.array(flux_norm_cont)

###    
# Plotting the original folded lightcurve
###  

    plt.figure(num=None, figsize=(pltSIZE/pltDPI, pltSIZE/pltDPI), dpi=pltDPI, facecolor=bg_color)

    plt.plot(phase_cont, flux_norm_cont, color=(R_color,G_color,B_color), marker='.', linestyle='none', markersize=0.7)
    plt.plot(phase_cont+1, flux_norm_cont, color=(R_color,G_color,B_color), marker='.', linestyle='none', markersize=0.7)

    plt.axis('off')

#     period_str = str(period).replace(".","_")
#     period_str = period_str[0:6]
#     filename = "T2CEP_LCs/" + var_Type + "_" + star_identifier + ".png"
    filename = "../testdata_color/" + var_Type + "_" + star_identifier + ".png"

#     filename = "T2CEP_LCs/" + period_str + "_" + var_Type + "_" + star_identifier + ".png"
#     filename = "T2CEP_LCs/" + period_str + ".png"
    print("Saving ORIGINAL", filename)
    plt.tight_layout(0)
    plt.gcf().savefig(filename, bbox_inches=None, pad_inches=0, facecolor=bg_color, edgecolor='none', dpi='figure')
#     plt.show()
    plt.close()
    
    parentCounter += 1

    print("=============================")
    print("Cycle:", row+1)
    print("Float ratio: ", floatRatio)
    print("Generated", cloneGenRatio, "clones for " + var_Type + "_" + star_identifier)
    print("Samples generated: ", parentCounter, "original", cloneCounter, "clone")
    print("=============================")


7000.652690000001 -- 1.8139278000000003
Max error: 0.031
Error array generated for t2cep_001
Clone generation 0
Saving ../testdata_color/t2cep_001_clone_0.png
Clone generation 1
Saving ../testdata_color/t2cep_001_clone_1.png
Clone generation 2
Saving ../testdata_color/t2cep_001_clone_2.png
Clone generation 3
Saving ../testdata_color/t2cep_001_clone_3.png
Clone generation 4
Saving ../testdata_color/t2cep_001_clone_4.png
Clone generation 5
Saving ../testdata_color/t2cep_001_clone_5.png
Clone generation 6
Saving ../testdata_color/t2cep_001_clone_6.png
Clone generation 7
Saving ../testdata_color/t2cep_001_clone_7.png
483
Saving ORIGINAL ../testdata_color/t2cep_001.png
Cycle: 1
Float ratio:  7.620689655172414
Generated 8 clones for t2cep_001
Samples generated:  1 original 8 clone
7014.1120599999995 -- 18.326155
Max error: 0.008
Error array generated for t2cep_002
Clone generation 0
Saving ../testdata_color/t2cep_002_clone_0.png
Clone generation 1
Saving ../testdata_color/t2cep_002_clone_1.p

722
Saving ORIGINAL ../testdata_color/t2cep_011.png
Cycle: 11
Float ratio:  7.608540925266904
Generated 8 clones for t2cep_011
Samples generated:  10 original 80 clone
7011.027290000001 -- 11.577937400000001
Max error: 0.009000000000000001
Error array generated for t2cep_012
Clone generation 0
Saving ../testdata_color/t2cep_012_clone_0.png
Clone generation 1
Saving ../testdata_color/t2cep_012_clone_1.png
Clone generation 2
Saving ../testdata_color/t2cep_012_clone_2.png
Clone generation 3
Saving ../testdata_color/t2cep_012_clone_3.png
Clone generation 4
Saving ../testdata_color/t2cep_012_clone_4.png
Clone generation 5
Saving ../testdata_color/t2cep_012_clone_5.png
Clone generation 6
Saving ../testdata_color/t2cep_012_clone_6.png
Clone generation 7
Saving ../testdata_color/t2cep_012_clone_7.png
718
Saving ORIGINAL ../testdata_color/t2cep_012.png
Cycle: 12
Float ratio:  7.607142857142857
Generated 8 clones for t2cep_012
Samples generated:  11 original 88 clone
7002.167559999999 -- 11.5313

Clone generation 2
Saving ../testdata_color/t2cep_021_clone_2.png
Clone generation 3
Saving ../testdata_color/t2cep_021_clone_3.png
Clone generation 4
Saving ../testdata_color/t2cep_021_clone_4.png
Clone generation 5
Saving ../testdata_color/t2cep_021_clone_5.png
Clone generation 6
Saving ../testdata_color/t2cep_021_clone_6.png
Clone generation 7
Saving ../testdata_color/t2cep_021_clone_7.png
699
Saving ORIGINAL ../testdata_color/t2cep_021.png
Cycle: 21
Float ratio:  7.594095940959409
Generated 8 clones for t2cep_021
Samples generated:  20 original 160 clone
7000.06831 -- 10.7069912
Max error: 0.01
Error array generated for t2cep_022
Clone generation 0
Saving ../testdata_color/t2cep_022_clone_0.png
Clone generation 1
Saving ../testdata_color/t2cep_022_clone_1.png
Clone generation 2
Saving ../testdata_color/t2cep_022_clone_2.png
Clone generation 3
Saving ../testdata_color/t2cep_022_clone_3.png
Clone generation 4
Saving ../testdata_color/t2cep_022_clone_4.png
Clone generation 5
Saving ..

7004.3723 -- 6.7079334
Max error: 0.012
Error array generated for t2cep_031
Clone generation 0
Saving ../testdata_color/t2cep_031_clone_0.png
Clone generation 1
Saving ../testdata_color/t2cep_031_clone_1.png
Clone generation 2
Saving ../testdata_color/t2cep_031_clone_2.png
Clone generation 3
Saving ../testdata_color/t2cep_031_clone_3.png
Clone generation 4
Saving ../testdata_color/t2cep_031_clone_4.png
Clone generation 5
Saving ../testdata_color/t2cep_031_clone_5.png
Clone generation 6
Saving ../testdata_color/t2cep_031_clone_6.png
Clone generation 7
Saving ../testdata_color/t2cep_031_clone_7.png
742
Saving ORIGINAL ../testdata_color/t2cep_031.png
Cycle: 31
Float ratio:  7.578544061302682
Generated 8 clones for t2cep_031
Samples generated:  30 original 240 clone
7003.025309999999 -- 44.40491420000001
Max error: 0.005
Error array generated for t2cep_032
Clone generation 0
Saving ../testdata_color/t2cep_032_clone_0.png
Clone generation 1
Saving ../testdata_color/t2cep_032_clone_1.png
Clo

Saving ../testdata_color/t2cep_040_clone_6.png
Clone generation 7
Saving ../testdata_color/t2cep_040_clone_7.png
728
Saving ORIGINAL ../testdata_color/t2cep_040.png
Cycle: 40
Float ratio:  7.563492063492063
Generated 8 clones for t2cep_040
Samples generated:  39 original 312 clone
7000.86362 -- 2.4760478999999997
Max error: 0.016
Error array generated for t2cep_041
Clone generation 0
Saving ../testdata_color/t2cep_041_clone_0.png
Clone generation 1
Saving ../testdata_color/t2cep_041_clone_1.png
Clone generation 2
Saving ../testdata_color/t2cep_041_clone_2.png
Clone generation 3
Saving ../testdata_color/t2cep_041_clone_3.png
Clone generation 4
Saving ../testdata_color/t2cep_041_clone_4.png
Clone generation 5
Saving ../testdata_color/t2cep_041_clone_5.png
Clone generation 6
Saving ../testdata_color/t2cep_041_clone_6.png
Clone generation 7
Saving ../testdata_color/t2cep_041_clone_7.png
728
Saving ORIGINAL ../testdata_color/t2cep_041.png
Cycle: 41
Float ratio:  7.561752988047809
Generated 

...

Clone generation 5
Saving ../testdata_color/t2cep_235_clone_5.png
Clone generation 6
Saving ../testdata_color/t2cep_235_clone_6.png
290
Saving ORIGINAL ../testdata_color/t2cep_235.png
Cycle: 235
Float ratio:  7.492063492063492
Generated 7 clones for t2cep_235
Samples generated:  228 original 1745 clone
7001.42805 -- 9.447274
Max error: 0.011000000000000001
Error array generated for t2cep_236
Clone generation 0
Saving ../testdata_color/t2cep_236_clone_0.png
Clone generation 1
Saving ../testdata_color/t2cep_236_clone_1.png
Clone generation 2
Saving ../testdata_color/t2cep_236_clone_2.png
Clone generation 3
Saving ../testdata_color/t2cep_236_clone_3.png
Clone generation 4
Saving ../testdata_color/t2cep_236_clone_4.png
Clone generation 5
Saving ../testdata_color/t2cep_236_clone_5.png
Clone generation 6
Saving ../testdata_color/t2cep_236_clone_6.png
Clone generation 7
Saving ../testdata_color/t2cep_236_clone_7.png
717
Saving ORIGINAL ../testdata_color/t2cep_236.png
Cycle: 236
Float ratio:  

Saving ../testdata_color/t2cep_245_clone_5.png
Clone generation 6
Saving ../testdata_color/t2cep_245_clone_6.png
495
Saving ORIGINAL ../testdata_color/t2cep_245.png
Cycle: 245
Float ratio:  7.490566037735849
Generated 7 clones for t2cep_245
Samples generated:  238 original 1820 clone
7002.952940000001 -- 6.2527840999999995
Max error: 0.017
Error array generated for t2cep_246
Clone generation 0
Saving ../testdata_color/t2cep_246_clone_0.png
Clone generation 1
Saving ../testdata_color/t2cep_246_clone_1.png
Clone generation 2
Saving ../testdata_color/t2cep_246_clone_2.png
Clone generation 3
Saving ../testdata_color/t2cep_246_clone_3.png
Clone generation 4
Saving ../testdata_color/t2cep_246_clone_4.png
Clone generation 5
Saving ../testdata_color/t2cep_246_clone_5.png
Clone generation 6
Saving ../testdata_color/t2cep_246_clone_6.png
Clone generation 7
Saving ../testdata_color/t2cep_246_clone_7.png
716
Saving ORIGINAL ../testdata_color/t2cep_246.png
Cycle: 246
Float ratio:  7.5
Generated 8 c

Clone generation 5
Saving ../testdata_color/t2cep_255_clone_5.png
Clone generation 6
Saving ../testdata_color/t2cep_255_clone_6.png
294
Saving ORIGINAL ../testdata_color/t2cep_255.png
Cycle: 255
Float ratio:  7.488372093023256
Generated 7 clones for t2cep_255
Samples generated:  248 original 1895 clone
7001.37583 -- 1.6201093
Max error: 0.047
Error array generated for t2cep_256
Clone generation 0
Saving ../testdata_color/t2cep_256_clone_0.png
Clone generation 1
Saving ../testdata_color/t2cep_256_clone_1.png
Clone generation 2
Saving ../testdata_color/t2cep_256_clone_2.png
Clone generation 3
Saving ../testdata_color/t2cep_256_clone_3.png
Clone generation 4
Saving ../testdata_color/t2cep_256_clone_4.png
Clone generation 5
Saving ../testdata_color/t2cep_256_clone_5.png
Clone generation 6
Saving ../testdata_color/t2cep_256_clone_6.png
Clone generation 7
Saving ../testdata_color/t2cep_256_clone_7.png
127
Saving ORIGINAL ../testdata_color/t2cep_256.png
Cycle: 256
Float ratio:  7.5
Generated 

Saving ORIGINAL ../testdata_color/t2cep_265.png
Cycle: 265
Float ratio:  7.484848484848484
Generated 7 clones for t2cep_265
Samples generated:  258 original 1970 clone
7000.21557 -- 5.0413596
Max error: 0.018000000000000002
Error array generated for t2cep_266
Clone generation 0
Saving ../testdata_color/t2cep_266_clone_0.png
Clone generation 1
Saving ../testdata_color/t2cep_266_clone_1.png
Clone generation 2
Saving ../testdata_color/t2cep_266_clone_2.png
Clone generation 3
Saving ../testdata_color/t2cep_266_clone_3.png
Clone generation 4
Saving ../testdata_color/t2cep_266_clone_4.png
Clone generation 5
Saving ../testdata_color/t2cep_266_clone_5.png
Clone generation 6
Saving ../testdata_color/t2cep_266_clone_6.png
Clone generation 7
Saving ../testdata_color/t2cep_266_clone_7.png
118
Saving ORIGINAL ../testdata_color/t2cep_266.png
Cycle: 266
Float ratio:  7.5
Generated 8 clones for t2cep_266
Samples generated:  259 original 1978 clone
7000.71215 -- 1.0311085
Max error: 0.091
Error array g

7010.46602 -- 32.128999199999996
Max error: 0.005
Error array generated for t2cep_276
Clone generation 0
Saving ../testdata_color/t2cep_276_clone_0.png
Clone generation 1
Saving ../testdata_color/t2cep_276_clone_1.png
Clone generation 2
Saving ../testdata_color/t2cep_276_clone_2.png
Clone generation 3
Saving ../testdata_color/t2cep_276_clone_3.png
Clone generation 4
Saving ../testdata_color/t2cep_276_clone_4.png
Clone generation 5
Saving ../testdata_color/t2cep_276_clone_5.png
Clone generation 6
Saving ../testdata_color/t2cep_276_clone_6.png
Clone generation 7
Saving ../testdata_color/t2cep_276_clone_7.png
696
Saving ORIGINAL ../testdata_color/t2cep_276.png
Cycle: 276
Float ratio:  7.5
Generated 8 clones for t2cep_276
Samples generated:  269 original 2053 clone
7013.585370000001 -- 30.3610752
Max error: 0.005
Error array generated for t2cep_277
Clone generation 0
Saving ../testdata_color/t2cep_277_clone_0.png
Clone generation 1
Saving ../testdata_color/t2cep_277_clone_1.png
Clone gener